<a href="https://colab.research.google.com/github/mekapothulapavani/FMML-LAB-ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

#1st Q


The accuracy of a machine learning model's validation set can be affected by the percentage of data allocated to the validation set. This allocation is typically determined by the train-validation-test split ratio, with the most common ratios being 70-30, 80-20, or 90-10. Here's how changing the percentage of the validation set can impact model validation:

Increase in the Percentage of Validation Set:

Pros:
More data in the validation set means you have a larger sample to evaluate the model's performance, which can lead to a more reliable estimation of how well the model generalizes to unseen data.
It can help in identifying overfitting. If the validation accuracy remains high even with a larger validation set, it suggests that the model might not be overfitting to the training data.
Cons:
As you increase the validation set size, the training set size decreases proportionally. With less data for training, the model might not learn as effectively, which can result in underfitting.
If your dataset is small to begin with, increasing the validation set percentage can lead to even smaller training sets, which may negatively impact the model's performance.
Reduction in the Percentage of Validation Set:

Pros:
A smaller validation set means more data is available for training, which can help the model learn better and potentially lead to improved performance.
If the dataset is large and the validation set is already sufficiently representative, reducing its size may not have a significant impact on the model's evaluation.
Cons:
With a smaller validation set, the evaluation of the model's performance may be less reliable. Validation accuracy can be sensitive to the specific data points in the validation set, and a small validation set might not provide a robust assessment of generalization.
It can be challenging to detect overfitting when the validation set is small. The model may perform well on the validation set by chance, even if it is overfitting the training data.
The choice of the percentage of the validation set should be based on the specific characteristics of your dataset and the goals of your machine learning project. A common practice is to start with a moderate split (e.g., 80-20) and then adjust it based on empirical results and the available data. Techniques like cross-validation can also be used to mitigate some of the issues associated with small validation sets.

In summary, increasing the percentage of the validation set can provide a more reliable estimate of model performance but may lead to smaller training sets and potential underfitting. Reducing the percentage of the validation set can result in better training but may yield less reliable model evaluation. The choice should be made carefully based on the trade-offs and requirements of your specific project.






In [ ]:
#2ND Q

The size of the train and validation sets can indeed affect how well we can predict the accuracy on the test set using the validation set. This relationship is influenced by several factors:

Sample Size and Representativeness: The accuracy estimated on the validation set is a statistical estimate of how well the model is likely to perform on unseen data, such as the test set. If the validation set is small or unrepresentative of the overall data distribution, its accuracy may not be a reliable predictor of the test set accuracy. In such cases, increasing the size of the validation set can lead to more reliable predictions.

Overfitting Detection: A larger validation set can provide a more robust signal for detecting overfitting. If the model is overfitting the training data, it's more likely to be identified when the validation set is sufficiently large. On the other hand, a small validation set may not capture the overfitting patterns effectively, leading to overly optimistic estimates of the model's performance.

Generalization Robustness: A smaller training set, resulting from a larger validation set, can potentially hinder the model's ability to generalize well. If the training set is too small, the model may not learn the underlying patterns in the data effectively, leading to poorer performance on both the validation and test sets.

Stability of Estimates: In statistical terms, as the sample size (validation set size) increases, the margin of error in estimating the model's accuracy decreases. This means that with a larger validation set, you're more likely to get a more stable and accurate estimate of the model's performance.

Resource Constraints: The choice of the train-validation split ratio may also be influenced by resource constraints such as computational power and time. Sometimes, practical limitations might necessitate a smaller validation set to allow for more extensive model experimentation.

In summary, the size of the train and validation sets can impact how well we can predict the accuracy on the test set using the validation set. Larger validation sets generally provide more reliable estimates of model performance, especially for detecting overfitting, but they may come at the cost of smaller training sets. The choice should be made based on the specific characteristics of your dataset, the goals of your project, and any resource constraints you might have. Cross-validation techniques can also be used to mitigate the impact of small validation sets on performance estimation.







#3RD Q

The accuracy of the validation set can be affected by the percentage of data allocated to the validation set. This allocation is typically determined by the train-validation-test split ratio, with common ratios being 70-30, 80-20, or 90-10. Here's how changing the percentage of the validation set can impact validation accuracy:

Increase in the Percentage of Validation Set:

Pros:
More data in the validation set means you have a larger sample to evaluate the model's performance, which can lead to a more reliable estimation of how well the model generalizes to unseen data.
It can help in identifying overfitting. If the validation accuracy remains high even with a larger validation set, it suggests that the model might not be overfitting to the training data.
Cons:
As you increase the validation set size, the training set size decreases proportionally. With less data for training, the model might not learn as effectively, which can result in underfitting.
If your dataset is small to begin with, increasing the validation set percentage can lead to even smaller training sets, which may negatively impact the model's performance.
Reduction in the Percentage of Validation Set:

Pros:
A smaller validation set means more data is available for training, which can help the model learn better and potentially lead to improved performance.
If the dataset is large and the validation set is already sufficiently representative, reducing its size may not have a significant impact on the model's evaluation.
Cons:
With a smaller validation set, the evaluation of the model's performance may be less reliable. Validation accuracy can be sensitive to the specific data points in the validation set, and a small validation set might not provide a robust assessment of generalization.
It can be challenging to detect overfitting when the validation set is small. The model may perform well on the validation set by chance, even if it is overfitting the training data.
The choice of the percentage of the validation set should be based on the specific characteristics of your dataset and the goals of your machine learning project. A common practice is to start with a moderate split (e.g., 80-20) and then adjust it based on empirical results and the available data. Techniques like cross-validation can also be used to mitigate some of the issues associated with small validation sets.

In summary, increasing the percentage of the validation set can provide a more reliable estimate of model performance but may lead to smaller training sets and potential underfitting. Reducing the percentage of the validation set can result in better training but may yield less reliable model evaluation. The choice should be made carefully based on the trade-offs and requirements of your specific project.







## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#1ST Q

Yes, averaging the validation accuracy across multiple splits of your data can provide more consistent and reliable results when evaluating the performance of a machine learning model. This technique is commonly known as cross-validation.

Cross-validation involves splitting the dataset into multiple subsets or "folds." The model is trained and validated multiple times, each time using a different fold as the validation set and the remaining folds as the training set. The results, such as accuracy, from each of these iterations are then averaged to obtain a more robust estimate of the model's performance. Common types of cross-validation include k-fold cross-validation and stratified k-fold cross-validation.

Here are some benefits of using cross-validation:

Reduced Variance: By averaging results across multiple validation sets, cross-validation reduces the variance in the performance estimate. This means that the evaluation is less dependent on the specific data points in a single validation set, making it a more stable measure of the model's generalization performance.

Better Detection of Overfitting: Cross-validation can provide a more accurate assessment of a model's tendency to overfit. If a model consistently performs




#2ND Q

Cross-validation does not directly give you a more accurate estimate of test accuracy; instead, it provides a more accurate estimate of how your model is likely to perform on unseen data. This estimate can be valuable because it helps you understand how well your model generalizes to new, unseen examples.

Here's how it works:

Training and Validation Within Cross-Validation: In a typical cross-validation procedure, your dataset is divided into multiple subsets or "folds." The model is trained on a combination of these folds (the training set) and evaluated on the remaining fold (the validation set). This process is repeated multiple times, with each fold serving as the validation set in turn. This results in multiple accuracy scores, one for each validation fold.

Averaging for Generalization Estimate: After all iterations of training and validation are completed, the accuracy scores obtained from each fold are averaged. This average accuracy is considered an estimate of how well your model is likely to perform on unseen data from the same population as your dataset.

Comparison to Test Set: While cross-validation provides a good estimate of generalization performance, the actual "test accuracy" is typically determined by evaluating the model on a separate, previously unseen test set. The test set is not used during model development, and its purpose is to simulate how well the model will perform in real-world scenarios with new data.

So, cross-validation doesn't directly give you the test accuracy but gives you a strong indication of how well your model is expected to perform on similar, unseen data. If your cross-validation results consistently show high accuracy, it suggests that your model is likely to perform well on the test set. However, the actual test accuracy may vary, especially if the test data is substantially different from the validation data.

In summary, cross-validation provides a more accurate estimate of how well your model generalizes to unseen data from the same population as your training and validation sets. While it's a valuable tool for model evaluation and selection, the true test accuracy is determined by evaluating the model on a separate, held-out test set that was not used during model development.






Regenerate


#3RD Q

The number of iterations in a cross-validation procedure can have an impact on the accuracy of the estimate of your model's performance. Generally, increasing the number of iterations (or folds) can lead to a more robust and reliable estimate, but it comes with trade-offs in terms of computational cost and variance in the estimate.

Here's how the number of iterations affects the estimate:

Fewer Iterations (Folds):

Pros:
Faster to compute since you are training and validating the model fewer times.
Requires less computational resources.
Cons:
The estimate may have higher variance because it's based on a smaller number of validation sets.
It may be sensitive to the specific data splits in each iteration.
More Iterations (Folds):

Pros:
Tends to provide a more stable and less variable estimate of model performance.
Reduces the sensitivity to the specific data splits because each data point participates in the validation set multiple times.
Cons:
Requires more computation time and resources as you train and validate the model more times.
In practice, the choice of the number of iterations (often referred to as "k" in k-fold cross-validation) should be a balance between computational resources and the desire for a reliable estimate. Common choices for k are 5-fold, 10-fold, and sometimes 3-fold or 4-fold.

Here are some considerations:

For small to moderately sized datasets, 5-fold or 10-fold cross-validation is often sufficient to obtain a reasonably stable estimate of model performance.

For very large datasets, you might consider using smaller values of k (e.g., 3-fold or 4-fold) to reduce computation time.

If computational resources are not a limiting factor, using a larger value of k can provide a more robust estimate of model performance.

Extremely large values of k (e.g., leave-one-out cross-validation, where k is equal to the number of data points) can be computationally expensive and may not always lead to a substantially better estimate, especially if the dataset is noisy.

In summary, increasing the number of iterations (folds) in cross-validation can generally provide a more stable and reliable estimate of model performance, but it also increases computational demands. The choice of the number of iterations should be based on practical considerations, such as the size of your dataset and available computational resources, while aiming for a balance between accuracy and efficiency.

#4TH Q

Increasing the number of iterations (folds) in cross-validation can help mitigate the impact of having a very small train or validation dataset to some extent, but it may not completely overcome the limitations of extremely small datasets. Here's how it works:

Benefits of Increasing Iterations:

When you have a very small dataset, each fold in cross-validation will also be small, which can lead to unstable and unreliable performance estimates.
Increasing the number of iterations (i.e., using a higher k value) means that you'll create more subsets for training and validation. This can help in reducing the variability in the performance estimates because each data point will have more chances to participate in the validation set.
It can provide a more robust estimate of how well the model generalizes, even when the individual subsets (folds) are small.
Limitations of Extremely Small Datasets:

While increasing iterations can improve stability, it doesn't fundamentally change the amount of data you have available for training and validation.
If your dataset is extremely small, even with many iterations, each fold may still be too small to effectively train a complex machine learning model. This can lead to underfitting and poor model performance.
Trade-offs:

Using a very high number of iterations (e.g., leave-one-out cross-validation) with an already small dataset can be computationally expensive and may not provide substantial benefits in terms of more reliable estimates.
There's a balance to strike between the number of iterations and the size of each fold. If you have a very small dataset, you might choose a moderate k value (e.g., 5-fold) to strike a balance between computational efficiency and the reliability of estimates.
In summary, increasing the number of iterations in cross-validation can help improve the reliability of performance estimates when dealing with small train or validation datasets. However, it cannot completely compensate for the limitations of extremely small datasets, and you may still encounter issues related to underfitting and limited data for training. The choice of the number of iterations should be made while considering the trade-offs between computational resources and the need for stable performance estimates.